In [51]:
import requests

In [52]:
from bs4 import BeautifulSoup
from currency_converter import CurrencyConverter
import geopandas as gpd
import pandas as pd

In [53]:
c = CurrencyConverter()

## Open Geojson

In [54]:
bulgaria_gdf = gpd.read_file("data/bg.json")

In [55]:
bulgaria_gdf["name"] = bulgaria_gdf["name"].str.lower()
bulgaria_gdf = bulgaria_gdf.replace("grad sofiya", "sofia cap.")

## Get Economic data

In [56]:
r = requests.get("https://www.nsi.bg/en/content/3930/statistical-regions-district")

In [57]:
soup = BeautifulSoup(r.text, "html.parser")

In [58]:
trs = soup.find("table").find_all("tr")[2:-1]

In [64]:
districts = []
for i, tr in enumerate(trs):
    tds = tr.find_all("td")
    region = tds[0].text.replace("\xa0", "").lower().strip()
    if region[0] != "-":
        print(region)
        continue
    else:
        region = region[1:]
    fields = [int(td.text.replace("\xa0", "")) for td in tr.find_all("td")[1:4]]
    total = round(sum(fields) / 3.0, 2)
    euros = c.convert(total, "BGN", "EUR")
    districts.append({"name": region, "wage": round(euros, 2)})

severozapaden
severen tsentralen
severoiztochen
yugoiztochen
yugozapaden
yuzhen tsentralen


In [65]:
wage_df = pd.DataFrame(districts)

## Merge data and export

In [68]:
wage_gdf = bulgaria_gdf.merge(wage_df, on="name", how="left")

In [69]:
wage_gdf.sort_values("wage", ascending=False)

,source,id,name,geometry,wage
27,https://simplemaps.com,BG22,sofia cap.,"POLYGON ((23.27851 42.53229, 23.26001 42.54813...",1731.26
17,https://simplemaps.com,BG03,varna,"POLYGON ((27.88396 42.84837, 27.87688 42.8478,...",1144.46
5,https://simplemaps.com,BG23,sofia,"POLYGON ((23.0595 42.70987, 23.04441 42.73961,...",1135.09
12,https://simplemaps.com,BG06,vratsa,"POLYGON ((23.97748 43.05988, 23.95909 43.03949...",1070.32
18,https://simplemaps.com,BG16,plovdiv,"POLYGON ((25.32954 42.12451, 25.30846 42.13136...",1051.06
19,https://simplemaps.com,BG24,stara zagora,"POLYGON ((26.13012 42.15704, 26.1296 42.10061,...",1048.51
10,https://simplemaps.com,BG18,ruse,"POLYGON ((26.33234 44.05493, 26.33455 44.04532...",1001.47
0,https://simplemaps.com,BG02,burgas,"POLYGON ((27.0479 42.08292, 27.04789 42.08309,...",990.04
13,https://simplemaps.com,BG15,pleven,"POLYGON ((24.02678 43.21173, 23.9594 43.23163,...",988.68
21,https://simplemaps.com,BG27,shumen,"POLYGON ((27.1191 42.95366, 27.07207 42.93839,...",982.04


In [70]:
wage_gdf = wage_gdf.to_crs(7803)
wage_gdf.to_file("data/bulgaria_avg_monthly_wage.gpkg")